In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.f1 = nn.Linear(1, 10)
        self.f2 = nn.Linear(10, 20)
        self.f3 = nn.Linear(20, 20)
        self.f4 = nn.Linear(20, 10)
        self.f5 = nn.Linear(10, 1)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [5]:
X = np.random.uniform(0, 1, size=(1000, 1))
# y = X[:, 0] - X[:, 1]**2 + 2 * X[:, 2] * X[:, 3] + (1 / ((1 + X[:, 4]) ** 6))
y = np.sinc(X).sum(axis=1, keepdims=True)

torch_data = TensorDataset(torch.Tensor(X), torch.Tensor(y))
data_loader = DataLoader(torch_data, batch_size=100)

In [7]:
model = Net()
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, model=model, use_diagonal=False)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] < all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.3495309054851532
epoch:  1, loss: 0.2029220163822174
epoch:  2, loss: 0.1240890845656395
epoch:  3, loss: 0.07414117455482483
epoch:  4, loss: 0.040942661464214325
epoch:  5, loss: 0.022917363792657852
epoch:  6, loss: 0.013786757364869118
epoch:  7, loss: 0.008482249453663826
epoch:  8, loss: 0.005538102239370346
epoch:  9, loss: 0.003943435847759247
epoch:  10, loss: 0.002971104346215725
epoch:  11, loss: 0.002390253124758601
epoch:  12, loss: 0.0019624277483671904
epoch:  13, loss: 0.0016213811468333006
epoch:  14, loss: 0.0013888555113226175
epoch:  15, loss: 0.0011850178707391024
epoch:  16, loss: 0.0010142086539417505
epoch:  17, loss: 0.0008790463325567544
epoch:  18, loss: 0.0007573597831651568
epoch:  19, loss: 0.0006532880943268538
epoch:  20, loss: 0.0005717977182939649
epoch:  21, loss: 0.0004979904624633491
epoch:  22, loss: 0.00043378854752518237
epoch:  23, loss: 0.0003801594430115074
epoch:  24, loss: 0.0003324049466755241
epoch:  25, loss: 0.00029556

In [ ]:
model = Net()
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=0.01, ld=1, model=model, use_diagonal=True)

all_loss = {}
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss[epoch+1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x)
        opt.update(loss)

        all_loss[epoch+1] += loss
    
    all_loss[epoch+1] /= len(data_loader)

    if epoch > 0 and all_loss[epoch] < all_loss[epoch+1]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch+1].detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.8393998146057129
epoch:  1, loss: 0.8350479006767273
epoch:  2, loss: 125.46124267578125
epoch:  3, loss: 756704.3125
epoch:  4, loss: 203142447104.0
epoch:  5, loss: 534557229056.0
epoch:  6, loss: 534760751104.0
epoch:  7, loss: 534822748160.0
epoch:  8, loss: 534839033856.0
epoch:  9, loss: 534839296000.0
epoch:  10, loss: 534839427072.0
epoch:  11, loss: 534839427072.0
epoch:  12, loss: 534839427072.0
epoch:  13, loss: 534839427072.0
epoch:  14, loss: 534839427072.0
epoch:  15, loss: 534839427072.0
epoch:  16
Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.


_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 23).